Please check out the original [tensorflow implementation](https://github.com/uclmr/emoji2vec) of Emoji2Vec by its authors as well as the [paper](https://arxiv.org/pdf/1609.08359.pdf) for more details. <br>
This notebook is intended to provide an intuition behind Emoji2Vec and provide an idea of its features rather than serve as a in-depth analysis of the framework.

In [48]:
# External dependencies
import os
import numpy as np
import pickle as pk
import gensim.models as gsm
import torch
from tabulate import tabulate

#for TSNE Visualization
import pandas as pd 
from sklearn.manifold import TSNE
import plotly.graph_objects as go

# Internal dependencies
from model import Emoji2Vec, ModelParams
from phrase2vec import Phrase2Vec
from utils import build_kb, get_examples_from_kb, generate_embeddings, get_metrics, generate_predictions
from visualize import visualize


### What is Emoji2Vec?
Emoji2Vec representations are a form of distributed representations for Unicode emojis, trained directly from natural language descriptions of emojis. <br>
This notebook presents a high level walkthrough the process of training Emoji2Vec representations and demonstrates some experimental results. <br><br>
First, lets define the logical steps in training Emoji2Vec representations.
1. For each emoji in a data set a number of natural language descriptions is collected.
2. Each description is encoded to a fixed form vector in a high dimensional space. 
    - Although it can be done by an arbitrary encoding method, this implementation follows the approach presented in the [paper](https://arxiv.org/pdf/1609.08359.pdf) using 300-dimensional [Google News word2vec embeddings](https://code.google.com/archive/p/word2vec/) together with a simple phrase encoder `phrase2vec.Phrase2Vec`.
3. A neural network model is trained to classify emojis from their descriptions.
    - Inside the model each unique emoji has its own vector of parameters that is updated when this emoji is being classified. Through continuous incrementation in training such vectors of parameters become emoji representations.
4. Neural network's parameters are extracted and used as distributed emoji representations that are embedded in the same space as the underlying word embeddings.

In [49]:
train = False

### How to train Emoji2Vec representations?
First, we need a couple of things to train Emoji2Vec representations. Most importantly we need something to encode natural language descriptions of emojis to a fixed form high dimensional vectors. For this we are going to use [Google News wor2vec embeddings](https://code.google.com/archive/p/word2vec/), so in order to continue with this notebook make sure you've downloaded those embeddings and placed a `.bin.gz` file in `data/word2vec`. <br><br>
Of course we also need some emojis and their natural language descriptions. Fortunately authors of the paper collected a data set that is part of this repository. <br>
Our training data set is in `data/training`. Let's add this directory to our parameters.

In [50]:
params = {"data_folder": "data/training"}

To train Emoji2Vec representations we need to put emojis in some sort of organised structure, so that after training we know which vector of parameters in the model is associated with which emoji. Of course the same needs to be done for each phrase (natural language description of emoji) in the data set so that we know which descrption describes which emoji.

In [51]:

print('reading training data from: ' + params["data_folder"])
kb, ind2phr, ind2emoji = build_kb(params["data_folder"])


reading training data from: data/training


We'll need this emoji mapping later, so for now let's save it to a file.

In [52]:
params.update({"mapping_file": "emoji_mapping_file.pkl"})

In [53]:
# Save the mapping from index to emoji
pk.dump(ind2emoji, open(params["mapping_file"], 'wb'))

Time to encode descriptions of emojis to fixed form vectors. As mentioned before we're using 300 dimensional Google News embeddings located in `data/word2vec`. As we will need them later we're going to save the generated embeddings of descriptions to a file `phrase_embeddings.pkl`. <br>Let's add those paths to our parameters.

In [54]:
params.update({"word2vec_embeddings_file": "data/word2vec/GoogleNews-vectors-negative300.bin.gz",
               "phrase_embeddings_file": "phrase_embeddings.pkl"})

To reitarete - phrase embeddings are embeddigns of each emoji description that will serve as input for the neural network, whereas Google News embeddings are what we use to generate phrase embeddings. <br>

In [55]:

# Generate embeddings for each phrase in the training set
embeddings_array = generate_embeddings(ind2phr=ind2phr, kb=kb, embeddings_file=params["phrase_embeddings_file"],
                                    word2vec_file=params["word2vec_embeddings_file"])

loading embeddings...


So we have the training input (phrase embeddings), now let's load our training and development data sets.

In [56]:
train_set = get_examples_from_kb(kb=kb, example_type='train')
dev_set = get_examples_from_kb(kb=kb, example_type='dev')

And define training parameters.

In [57]:

model_params = ModelParams(in_dim=300, 
                        out_dim=300, 
                        max_epochs=60, 
                        pos_ex=4, 
                        neg_ratio=1, 
                        learning_rate=0.001,
                        dropout=0.0, 
                        class_threshold=0.5)

Define the model, it needs to know the training parameters, the number of emojis (size of parameter matrix depends on it) and the matrix of phrase embeddings (embeddings of emoji descriptions).

In [58]:

model = Emoji2Vec(model_params=model_params, num_emojis=kb.dim_size(0), embeddings_array=embeddings_array)

Now that the model has been defined, let's train it! 

In [59]:
if train :
    model.train(kb=kb, epochs=model_params.max_epochs, learning_rate=model_params.learning_rate)

Let's save the trained PyTorch model to a file so that we can load it later if we need it.

In [60]:
if train :
    model_folder = 'example_emoji2vec'
    if not os.path.isdir(model_folder):
        os.makedirs(model_folder)
    
    torch.save(model.nn, model_folder + '/model.pt')

In [61]:
if train==False :
    model_folder = 'example_emoji2vec'
    model.nn = torch.load(model_folder + '/model.pt')


Now that the model is trained and saved we need to extract emoji representations from the neural network's parameters. <br>
This can be done by a `model.Emoji2Vec` method called `create_gensim_files`, which will save the distributed representations of emojis in a format compatible with `gensim.models` allowing us to add Emoji2Vec representations to a gensim model and use them as any other word embeddings.


In [62]:
e2v = model.create_gensim_files(model_folder=model_folder, ind2emoj=ind2emoji, out_dim=model_params.out_dim)

#### Done.
That's it, you've generated your own Emoji2Vec representations which now sit in `example_emoji2vec/` as `emoji2vec.txt` and `emoji2vec.bin`. Good job. <br>
Now is the time for the fun part, for example we can create a gensim model with our newly generated emoji embeddings.

In [63]:
e2v = gsm.KeyedVectors.load_word2vec_format("example_emoji2vec/emoji2vec.bin", binary=True)

Let's take a look at some of the emojis in the data set.

In [64]:
vocabulary = [key for key in e2v.key_to_index]

In [65]:
# Sample 10 random emojis from the data set.
example_emojis = np.random.choice(list(vocabulary), 10)
print(example_emojis)

['👜' '📕' '🇬🇷' '⛅️' '📫' '♐' '🐤' '🏉' '🇬🇾' '🔣']


Each of the above emojis has its own vector representation.

In [66]:
e2v[example_emojis]

array([[-0.3679468 ,  0.00168402, -0.4711585 , ...,  0.11752382,
        -0.4276958 , -0.12119073],
       [-0.15438907, -0.4375898 , -0.09648127, ...,  0.6098831 ,
        -0.01111347,  0.03629427],
       [-0.00474668,  0.13680199,  0.19537875, ..., -0.08347146,
        -0.1081543 , -0.02951165],
       ...,
       [-0.85650986, -0.5421017 ,  0.12751018, ...,  0.03175882,
        -0.6157671 , -0.05484686],
       [-0.21084985, -0.24620818,  0.0832018 , ..., -0.01567112,
         0.09533603, -0.05419327],
       [ 0.22814782, -0.29438126, -0.23147367, ..., -0.41529545,
         0.14431834, -0.06887833]], dtype=float32)

Thanks to the fact that the learnt emoji representations are compatible with `gensim.models`, we can do a lot of cool things like finding similar emojis.

In [67]:
e2v.most_similar('🚔')

[('🚓', 0.8738017082214355),
 ('🚑', 0.6391749382019043),
 ('🚗', 0.5790260434150696),
 ('👮🏿', 0.5675467252731323),
 ('🚘', 0.5655882358551025),
 ('🚐', 0.5369750261306763),
 ('🚙', 0.5357747673988342),
 ('👮', 0.5355160236358643),
 ('🚃', 0.525036633014679),
 ('🚒', 0.5179762244224548)]

In [68]:
e2v.most_similar('🏊')

[('🦋', 0.5947760939598083),
 ('🛀', 0.5851469039916992),
 ('🌊', 0.5719010233879089),
 ('🏊🏻', 0.5664109587669373),
 ('🤾', 0.5615712404251099),
 ('🐃', 0.5562608242034912),
 ('🚰', 0.5495843887329102),
 ('🏊🏼', 0.5461294054985046),
 ('🚿', 0.5455986857414246),
 ('💦', 0.5421949028968811)]

In [69]:
e2v.most_similar('🚄')

[('🚅', 0.8224267363548279),
 ('🚋', 0.680420994758606),
 ('🚆', 0.678656280040741),
 ('🚃', 0.651321530342102),
 ('🚉', 0.6461228728294373),
 ('🚟', 0.6257645487785339),
 ('🚈', 0.6000259518623352),
 ('🚂', 0.5937529802322388),
 ('🚠', 0.5721362829208374),
 ('🚝', 0.5674235224723816)]

In [70]:
e2v.most_similar('😂')

[('😹', 0.726284921169281),
 ('😆', 0.6457535028457642),
 ('😢', 0.6286451816558838),
 ('😊', 0.6132701635360718),
 ('😭', 0.5730255246162415),
 ('😀', 0.5688401460647583),
 ('☺️', 0.561689019203186),
 ('😨', 0.5600235462188721),
 ('😃', 0.55708909034729),
 ('🙂', 0.5358986258506775)]

## 🤴 - 👨 + 👩 (= 👸)

In [71]:
e2v.most_similar(positive=['👩', '🤴'], negative=['👨'])

[('👑', 0.5246703028678894),
 ('👸', 0.5064293742179871),
 ('👸🏼', 0.456595778465271),
 ('💂', 0.42242008447647095),
 ('💏', 0.40975672006607056),
 ('🇮🇨', 0.4070007801055908),
 ('💋', 0.40582364797592163),
 ('🤰', 0.40565934777259827),
 ('📿', 0.405147910118103),
 ('👯', 0.40252700448036194)]

In [72]:
visualize(model)

usage: ipykernel_launcher.py [-h] [-d DIR] [-w WORD] [-m MAPPING]
                             [-em EMBEDDINGS] [-k DIM] [-b BATCH] [-e EPOCHS]
                             [-r RATIO] [-l LEARNING] [-dr DROPOUT]
                             [-t THRESHOLD] [-ds DATASET] [-D DEBUG]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9035 --control=9033 --hb=9032 --Session.signature_scheme="hmac-sha256" --Session.key=b"927b071a-a9e0-47be-8e21-6dd60b86a55e" --shell=9034 --transport="tcp" --iopub=9036 --f=/home/gchenu/.local/share/jupyter/runtime/kernel-v2-18823dtYtlKU0cwq.json


SystemExit: 2

However the real fun starts when we combine the power of emoji embeddings with word embeddings. <br>
Let's create a model that combines the two, this will allow us to measure similarity between emojis and phrases. 

In [ ]:
phraseVecModel = Phrase2Vec.from_word2vec_paths(300,
                                                "data/word2vec/GoogleNews-vectors-negative300.bin.gz",
                                                "example_emoji2vec/emoji2vec.bin")

In [ ]:
# mapping from id to emoji
mapping = pk.load(open(params["mapping_file"], 'rb'))
# mapping from emoji to id
inverse_mapping = {v: k for k, v in mapping.items()}

In [ ]:
def find_analogous_emoji(emoji_1,
                         emoji_2,
                         emoji_3,
                         top_n,
                         mapping=mapping,
                         inverse_mapping=inverse_mapping,
                         e2v=e2v,
                         model=model):
    similarities = []
    vector = e2v[emoji_1] - e2v[emoji_2] + e2v[emoji_3]
    vector = vector / np.linalg.norm(vector)
    
    for idx in range(len(mapping)):
        emoij_idx_similarity = model.nn.forward(torch.Tensor(vector.reshape(1, -1)), idx).detach().numpy()
        similarities.append(emoij_idx_similarity)
    
    similarities = np.array(similarities)
    n_most_similar_idxs = similarities.argsort(axis=0)[-top_n:][::-1].reshape(-1)
    n_most_similar_emojis = [mapping[emoji_idx] for emoji_idx in n_most_similar_idxs]
    
    str_expression = ' '.join([emoji_1, "-", emoji_2, "+", emoji_3,])
    top_score = similarities[n_most_similar_idxs[0]]
    
    return str_expression, n_most_similar_emojis
        

In [ ]:
table = [["Expression", "Closest emojis"],
         [*find_analogous_emoji("🤴", "🚹", "🚺", 3)],
         [*find_analogous_emoji("👑", "🚹", "🚺", 3)],
         [*find_analogous_emoji("👦", "🚹", "🚺", 3)],
         [*find_analogous_emoji("💵", "🇺🇸", "🇬🇧", 3)],
         [*find_analogous_emoji("💵", "🇺🇸", "🇪🇺", 3)],
         [*find_analogous_emoji("💷", "🇬🇧", "🇪🇺", 3)]]

In [ ]:
print(tabulate(table, headers="firstrow", tablefmt="rst"))

============  ==================
Expression    Closest emojis
============  ==================
🤴 - 🚹 + 🚺  ['🚺', '👸', '👪']
👑 - 🚹 + 🚺  ['👑', '👸', '🏰']
👦 - 🚹 + 🚺  ['👪', '🚺', '👸']
💵 - 🇺🇸 + 🇬🇧  ['💵', '💷', '💴']
💵 - 🇺🇸 + 🇪🇺  ['💵', '💷', '💶']
💷 - 🇬🇧 + 🇪🇺  ['💷', '💶', '💵']
============  ==================


Although far from perfect, similarly to the [published](https://arxiv.org/pdf/1609.08359.pdf) results an emoji considered to be "right" is usually within the top 3 examples selected by the model.

Combining Emoji2Vec representations with word2vec embeddings they were trained on also allows to inspect the relationships between word embeddings and emoji embeddings. For example with the `phrase2emoji` function below we can see emojis most similar to an arbitrary phrase.

In [ ]:
def phrase2emoji(phrase, 
                 top_n,
                 phraseVecModel=phraseVecModel,
                 mapping=mapping,
                 model=model):
    
    similarities = []
    phrase_vec = phraseVecModel[phrase]
    for idx in range(len(mapping)):
        emoij_idx_similarity = model.nn.forward(torch.Tensor(phrase_vec.reshape(1, -1)), idx).detach().numpy()
        similarities.append(emoij_idx_similarity)
        
    similarities = np.array(similarities)
    n_most_similar_idxs = similarities.argsort(axis=0)[-top_n:][::-1].reshape(-1)
    n_most_similar_emojis = [mapping[emoji_idx] for emoji_idx in n_most_similar_idxs]
    
    return phrase, n_most_similar_emojis
    

In [ ]:
table = [["Phrase", "Closest emojis"],
         [*phrase2emoji("minecraft", 3)],
         [*phrase2emoji("meme", 3)],
         [*phrase2emoji("a good project rating", 3)],
         [*phrase2emoji("natural language processing", 3)],
         [*phrase2emoji("the school project", 3)]]

In [ ]:
print(tabulate(table, headers="firstrow", tablefmt="rst"))

===========================  ========================
Phrase                       Closest emojis
===========================  ========================
minecraft                    ['🙏🏿', '🐏', '💸']
meme                         ['💬', '👅', '💨']
a good project rating        ['💯', '👍', '🇮🇸']
natural language processing  ['🙏🏿', '👲🏽', '👆🏾']
the school project           ['🚌', '🏫', '🚍']
===========================  ========================


There seems to exist a strong relationship between simple adjectives and emojis. That's great, but is this the whole point of Emoji2Vec? <br>
Of course not. In fact Emoji2Vec authors evaluated their method's performance on a downstream task of sentiment classification on a dataset  by  Kralj   Novak  et  al.  (2015),  which  consists of over 67k English tweets labelled manually for positive, neutral, or negative sentiment. Using Emoji2Vec alongside word embeddings, as opposed to using just word embeddings, yielded an improvement in classification accuracy across all studied datasets. In the same task Emoji2Vec also outperformed an alternative method for emoji representation. <br>
For more details check out the [paper](https://arxiv.org/pdf/1609.08359.pdf). You can also inspect the results of the sentiment classification task in [this notebook](https://github.com/uclmr/emoji2vec/blob/master/TwitterClassification.ipynb).